<img src="../images/BDG_LOGO.png" alt="drawing" align="right" width="200"/>

# H2020 RIA BigDataGrapes - Predictive Data Analytics (T4.3)

### This jupyter notebook is described in the deliverable D4.3 (Pilot 4). 
### In this pilot, we investigate how the biological activity depends on the location of the vineyard, the agriculture practices followed, the extraction method used, and the variety of the grape. The data collected from the natural cosmetics pilot provides the necessary information for the evaluation of the quality of each sample, linked with the special characteristics of the vineyard of origin.

In [1]:
!pip install --user scipy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
import scipy
import numpy as np
import pandas as pd
import datetime
import pickle

ImportError: No module named pandas

## Parameters
1. year_of_analysis: inform for what year the analysis will be performed.
2. type analysis: which Biological Activity (BA): Marceration or Ultrasound.
3. satellite_source: which satellite source has been used

In [ ]:
year_of_analysis = 2018
type_analysis = 'Maceration' # Biological Activity (BA): Maceration or Ultrasound
satellite_source = 'sentinel2' # Source: 'sentinel2' or 'landsat8'

## Loading Sample Data

In [ ]:
with open('../datasets/maceration_ultrasound/sample.pickle', 'rb') as file:
    exclude_sample = [142335]
    
    dict_sample = pickle.load(file)
    df_samples = dict_sample[str(year_of_analysis)]

    # filter the ones without a parcel_id
    df_samples = df_samples[df_samples['geocledian_id'].isna() == False]
    df_samples = df_samples[df_samples['geocledian_id'].isin(exclude_sample) == False]
    df_samples['geocledian_id'] = df_samples['geocledian_id'].astype(int)

print(len(df_samples))
df_samples


## Loading Maceration or Ultrasound Data

The Biologic Activity data, wich can be Maceration and Ultrasound.

In [ ]:
# open a file, where you stored the pickled data
with open('../datasets/maceration_ultrasound/ba_results_2018.pickle', 'rb') as file:

    dict_ba = pickle.load(file)
    df_ba = dict_ba[type_analysis]
    
    # drop some columns
    df_ba.drop("Total microbial count", axis=1, inplace=True)
    df_ba.drop("Toxicity on skin cells (MTT assay)", axis=1, inplace=True)
    df_ba.drop("Gene expression (SIRT1) on skin cells", axis=1, inplace=True)
    df_ba.drop("Yeasts and moulds", axis=1, inplace=True)
    df_ba.drop("year", axis=1, inplace=True)

    list_of_maceration_properties = df_ba.columns
    

## Merging: Maceration & Sample Data

In [ ]:
df_map = df_samples[['geocledian_id', 'sample_id']]
df_ba = df_ba.merge(df_map, left_on="Sample_Id", right_on="sample_id")


## Loading Parcels Satellite Data

Satellite Imagery data, more specifically the vegetation indexes.

In [ ]:

def adjust_dataframe_float_types(df, column_name):
    df[column_name].replace('None', np.nan, inplace=True)
    return df[column_name].astype('float64')


columns = ["key", "key_name", "source", "product", "parcel_id", "date", "min", "max", "mean", "std", "count", "sum", "index"]
df_parcels = pd.read_csv('../datasets/agknow/agknow_timeseries_BDG_apikey.csv', sep=';', names=columns, parse_dates=['date'])

for col_name in ['max', 'min', 'mean', 'std', 'count', 'sum']: 
    df_parcels[col_name] = adjust_dataframe_float_types(df_parcels, col_name)



## Merging: Maceration Data & Geocledian Data

In [ ]:
list_of_parcel_ids = list(df_parcels['parcel_id'].unique())
lst_maceration_parcel_ids = df_ba['geocledian_id'].tolist()

df_ba = df_ba[df_ba["geocledian_id"].isin(list_of_parcel_ids)]
df_parcels = df_parcels[df_parcels['parcel_id'].isin(lst_maceration_parcel_ids)]

list_of_parcel_ids = list(df_parcels['parcel_id'].unique())
list_of_product_types = list(df_parcels['product'].unique())

print (list_of_parcel_ids)
print (list_of_product_types)

## Creating Indexes 

### Index: Samples to parcels, parcels to samples.

In [ ]:
# df_samples
sample_to_parcel = {}
parcel_to_sample = {}

for idx, r in df_samples[["sample_id", "geocledian_id"]].iterrows():
    sample_to_parcel[str(r["sample_id"])] = str(r["geocledian_id"])
    parcel_to_sample[str(r["geocledian_id"])] = str(r["sample_id"])

print("sample_to_parcel", sample_to_parcel)
print("parcel_to_sample", parcel_to_sample)

### Index: Parcel_id to Crop_date

In [ ]:
parcel_date_crop_dict = {}

for parcel_id in list_of_parcel_ids:
    date_crop = df_samples[df_samples["geocledian_id"] == parcel_id]["Sample collection day"].max()
    
    date_crop = datetime.datetime(date_crop.year, date_crop.month, date_crop.day)
    parcel_date_crop_dict[parcel_id] = date_crop

parcel_date_crop_dict

### Index: BA_property -> [Property_value]

In [ ]:
def get_ba_property_dict():

    maceration_property_dict = {}

    for prop in list_of_maceration_properties:
        if prop in ['Sample', 'Sample_Id']:
            continue
        maceration_property_dict[prop] = []
        for parcel_id in list_of_parcel_ids:
            sample_id = parcel_to_sample[str(parcel_id)]
            df_parcel_maceration = df_ba[df_ba['sample_id'] == sample_id]
            maceration_property_dict[prop].append(float(df_parcel_maceration[prop]))

    # print(maceration_property_dict)
    return maceration_property_dict

### Index: Parcel_id -> Agg_function -> [product_type value]

In [ ]:
 
def get_product_type_signals_dict(agg_value_column, agg_time_function, satellite_source):
    product_type_signals_dict = {}

    for product_type in list_of_product_types:
        if product_type == 'variations':
            continue

        product_type_signals_dict[product_type] = []
        for parcel_id in list_of_parcel_ids:
            # properties filter
            df_parcels_filtered = df_parcels[(df_parcels['parcel_id'] == parcel_id) & (df_parcels['product'] == product_type) & (df_parcels['source'] == satellite_source)]
            # exclude non
            df_parcels_filtered = df_parcels_filtered[df_parcels_filtered[agg_value_column].notna()]
            # getting the start and the end dates
            
            # if theere is no information to compute 
            if len(df_parcels_filtered) == 0:
                # continue
                raise Exception ("Error: No parcel values!")
            
            df_parcels_filtered = agg_time_function([parcel_id, df_parcels_filtered, agg_value_column])
 
            # series 
            se_product_type_agg_value = df_parcels_filtered[agg_value_column]
            se_product_type_agg_value = pd.to_numeric(pd.Series(se_product_type_agg_value), errors='coerce')
            product_type_signals_dict[product_type].append(se_product_type_agg_value.mean())

    return product_type_signals_dict


### Index - Maceration/Ultrasound Property Abreviation

In [ ]:

dict_abreviation_maceration_property = {
    'TFC': 'Total flavonoid content, TFC (μg/mL quercetin)',
    'ABTS': 'Antioxidant activity ABTS (μg/mL trolox)',
    'Refractive': 'Refractive Index %',
    'TPC': 'Total phenolic content, TPC (μg/mL gallic acid)',
    'pH':'pH',
    'DPPH': 'Antioxidant activity DPPH (μg/mL trolox)'     
}


## Correlation Matrix Function

The coefficient returns a value between -1 and 1 that represents the limits of correlation from a full negative correlation to a full positive correlation. 
* A value of 0 means no correlation;
* The value must be interpreted, where often a value below -0.5 or above 0.5 indicates a notable correlation;
* Values below those values suggest a less notable correlation.

In [ ]:

def fn_correlation_matrix(agg_value_column, agg_time_function, satellite_source, verbose=False):
    
    product_type_signals_dict = get_product_type_signals_dict(agg_value_column, agg_time_function, satellite_source)
    ba_property_dict = get_ba_property_dict()
    
    # build the correlation matrix
    correlation_matrix = np.ones((len(ba_property_dict.keys()), len(product_type_signals_dict.keys())), dtype=np.float64)
    
    for i, observed_variable in enumerate(ba_property_dict.keys()):
        for j, current_signal in enumerate(product_type_signals_dict.keys()):
            np_observed_array = np.array(ba_property_dict[observed_variable])
            np_signal_array = np.array(product_type_signals_dict[current_signal])
            
            # np.corrcoef: returns pearson product-moment correlation coefficients.
            current_correlation = np.corrcoef(np_observed_array, np_signal_array)[0,1]
            correlation_matrix[i][j] = current_correlation
            if verbose: 
                print("correlation between {} and {} {}: {}".format(observed_variable, current_signal, agg_value_column, current_correlation))    
    
    
    y_labels = ['TFC', 'ABTS', 'Refractive', 'TPC', 'pH', 'DPPH'] # maceration_property_dict.keys()
    x_labels = product_type_signals_dict.keys()
    
    return correlation_matrix, x_labels, y_labels



## Auxiliar functions: Plots

In [ ]:
from dateutil import relativedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


def plot_heat_map(correlation_matrix, x_labels, y_labels, title, plot_shape, idx_plot):        
    plt.subplot(plot_shape[0], plot_shape[1],idx_plot)
    ax = sns.heatmap(correlation_matrix, linewidth=0.5, center=0, cmap="RdBu")
    ax.set_title(title)
    ax.set_yticklabels(y_labels, rotation=0, fontsize="10", va="center")
    ax.set_xticklabels(x_labels, rotation=0, fontsize="10", va="center")


def get_the_most_correlated_variables(correlation_matrix, x_labels, y_labels):
    abs_matrix = np.abs(correlation_matrix)
    max_corr_idx = np.argmax(abs_matrix, axis=1)
    results = []

    for i,idx_max in enumerate(max_corr_idx):
        item = (y_labels[i], list(x_labels)[idx_max], correlation_matrix[i][idx_max], abs_matrix[i][idx_max])
        results.append(item)

    df = pd.DataFrame(results,  columns=[type_analysis,'Product_Type','Highest_Corr_Value','Highest_Corr_Value_Abs'])
    return df


### Time aggregation functions

In [ ]:


def interval_from_the_begging_of_the_year_until_march(params_list, verbose=False):
    parcel_id = params_list[0]
    df_timeseries = params_list[1]
    
    date_crop = parcel_date_crop_dict[parcel_id]
    date_end = datetime.datetime(date_crop.year, month=3, day=31)
    date_start = datetime.datetime(date_crop.year, month=1, day=1)
    
    # temporal filter
    df_filtered = df_timeseries[(df_timeseries.date >= date_start) & (df_timeseries.date <= date_end)]
    
    if verbose:
        print ("parcel_id {} start {} end {} crop {} len {}".format(parcel_id, date_start.strftime("%d-%m-%Y"), date_end.strftime("%d-%m-%Y"), date_crop.strftime("%d-%m-%Y"), len(df_filtered)))
    
    return df_filtered


def interval_from_the_begging_of_the_year_until_april(params_list, verbose=False):
    parcel_id = params_list[0]
    df_timeseries = params_list[1]
    
    date_crop = parcel_date_crop_dict[parcel_id]
    date_end = datetime.datetime(date_crop.year, month=4, day=30)
    date_start = datetime.datetime(date_crop.year, month=1, day=1)
    
    # temporal filter
    df_filtered = df_timeseries[(df_timeseries.date >= date_start) & (df_timeseries.date <= date_end)]
    
    if verbose:
        print ("parcel_id {} start {} end {} crop {} len {}".format(parcel_id, date_start.strftime("%d-%m-%Y"), date_end.strftime("%d-%m-%Y"), date_crop.strftime("%d-%m-%Y"), len(df_filtered)))
    
    return df_filtered


def interval_from_the_begging_of_the_year_until_may(params_list, verbose=False):
    parcel_id = params_list[0]
    df_timeseries = params_list[1]
    
    date_crop = parcel_date_crop_dict[parcel_id]
    date_end = datetime.datetime(date_crop.year, month=5, day=31)
    date_start = datetime.datetime(date_crop.year, month=1, day=1)
    
    # temporal filter
    df_filtered = df_timeseries[(df_timeseries.date >= date_start) & (df_timeseries.date <= date_end)]
    
    if verbose:
        print ("parcel_id {} start {} end {} crop {} len {}".format(parcel_id, date_start.strftime("%d-%m-%Y"), date_end.strftime("%d-%m-%Y"), date_crop.strftime("%d-%m-%Y"), len(df_filtered)))
    
    return df_filtered

def interval_from_the_begging_of_the_year_until_june(params_list, verbose=False):
    parcel_id = params_list[0]
    df_timeseries = params_list[1]
    
    date_crop = parcel_date_crop_dict[parcel_id]
    date_end = datetime.datetime(date_crop.year, month=6, day=30)
    date_start = datetime.datetime(date_crop.year, month=1, day=1)
    
    # temporal filter
    df_filtered = df_timeseries[(df_timeseries.date >= date_start) & (df_timeseries.date <= date_end)]
    
    if verbose:
        print ("parcel_id {} start {} end {} crop {} len {}".format(parcel_id, date_start.strftime("%d-%m-%Y"), date_end.strftime("%d-%m-%Y"), date_crop.strftime("%d-%m-%Y"), len(df_filtered)))
    
    return df_filtered


## RQ. How correlated are the satellite imagery indexes and the biologicy activity properties?

The analytic API performs associations and correlations between the biologic activity properties and satellite imagery indexes (satellite data layer).

In [ ]:
###  @parameters 
list_of_agg_value_columns = ['max', 'min', 'mean', 'std', 'count', 'sum']
list_of_agg_time_functions = [interval_from_the_begging_of_the_year_until_march, 
                interval_from_the_begging_of_the_year_until_april, 
                interval_from_the_begging_of_the_year_until_may, 
                interval_from_the_begging_of_the_year_until_june]

for agg_time_function in list_of_agg_time_functions:
    plt.figure(figsize=(24,10))
    idx_plot = 1
    print(agg_time_function.__name__)
    for agg_value_column in list_of_agg_value_columns:
        correlation_matrix, x_labels, y_labels = fn_correlation_matrix(agg_value_column, agg_time_function, satellite_source, verbose=False)
        # plot 
        plot_title = "Agg.: {}. Intrv: {}".format(agg_value_column, agg_time_function.__name__.replace("interval_", ""))        
        plot_heat_map(correlation_matrix, x_labels, y_labels, plot_title, (2,3), idx_plot)
        idx_plot = idx_plot + 1
        
    plt.show()


## RQ. What are the most correlated satellite imagery indexes for each maceration property?

In [ ]:
list_of_agg_value_columns = ['max', 'min', 'mean', 'std', 'count', 'sum']
list_of_agg_time_functions = [interval_from_the_begging_of_the_year_until_march, 
                interval_from_the_begging_of_the_year_until_april, 
                interval_from_the_begging_of_the_year_until_may, 
                interval_from_the_begging_of_the_year_until_june]

df_highest_corr = None
for agg_value_column in list_of_agg_value_columns:
    for agg_time_function in list_of_agg_time_functions:
        correlation_matrix, x_labels, y_labels = fn_correlation_matrix(agg_value_column, agg_time_function, satellite_source, verbose=False) # fn_correlation_matrix(agg_function, satellite_source, days_before_the_crop, verbose=False)
        df_result = get_the_most_correlated_variables(correlation_matrix, x_labels, y_labels)
        df_result['Agg_value_column'] = agg_value_column
        df_result['Agg_time_function'] = agg_time_function.__name__
        df_result['Source'] = satellite_source
        
        if df_highest_corr is None:
            df_highest_corr = df_result
        else:
            df_highest_corr = pd.concat([df_highest_corr, df_result])

                        
pd.set_option('max_colwidth', 800)
df_highest_corr_by_ba = df_highest_corr.sort_values(by=[type_analysis, 'Highest_Corr_Value_Abs'], ascending=False)
df_highest_corr_by_ba = df_highest_corr_by_ba.drop_duplicates(subset=[type_analysis], keep='first')
df_highest_corr_by_ba.rename(columns={'Product_Type':'Most_correlated_Product_Type'}, inplace=True)
df_highest_corr_by_ba[[type_analysis, 'Most_correlated_Product_Type', 'Highest_Corr_Value', 'Highest_Corr_Value_Abs','Agg_time_function', 'Agg_value_column', 'Source']]
df_highest_corr_by_ba = df_highest_corr_by_ba[[type_analysis, 'Most_correlated_Product_Type', 'Highest_Corr_Value', 'Highest_Corr_Value_Abs','Agg_time_function', 'Agg_value_column']]
df_highest_corr_by_ba
            

## RQ. What are the most correlated maceration properties for each satellite imagery index?

In [ ]:
pd.set_option('max_colwidth', 800)

df_highest_corr_by_Satellity_Index = df_highest_corr.sort_values(by=['Product_Type', 'Highest_Corr_Value_Abs'], ascending=False)
df_highest_corr_by_Satellity_Index = df_highest_corr_by_Satellity_Index.drop_duplicates(subset=['Product_Type'], keep='first')
df_highest_corr_by_Satellity_Index.rename(columns={type_analysis:'Most_correlated_{0}_Prop'.format(type_analysis)}, inplace=True)
df_highest_corr_by_Satellity_Index[['Product_Type', 'Most_correlated_{0}_Prop'.format(type_analysis), 'Highest_Corr_Value', 'Highest_Corr_Value_Abs', 'Agg_time_function', 'Agg_value_column', 'Source']]


df_highest_corr_by_Satellity_Index = df_highest_corr_by_Satellity_Index[['Product_Type', 'Most_correlated_{0}_Prop'.format(type_analysis), 'Highest_Corr_Value', 'Highest_Corr_Value_Abs', 'Agg_time_function', 'Agg_value_column']]
df_highest_corr_by_Satellity_Index